In [0]:
# Load data into dataframe

df_oct = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv", header=True, inferSchema=True)
display(df_oct.limit(2))
df_nov = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv", header=True, inferSchema=True)
display(df_nov.limit(2))

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
2019-10-01T00:00:00.000Z,view,44600062,2103807459595387724,null,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
2019-10-01T00:00:00.000Z,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.2,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc


event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
2019-11-01T00:00:00.000Z,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
2019-11-01T00:00:00.000Z,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283


In [0]:
df_joined = df_oct.alias("o").join(df_nov.alias("n"), on=["product_id","user_id"], how="left") \
    .filter("o.event_type = 'purchase'")
display(df_joined.limit(5))

product_id,user_id,event_time,event_type,category_id,category_code,brand,price,user_session,event_time,event_type,category_id,category_code,brand,price,user_session
1005108,555394812,2019-10-31T00:33:37.000Z,purchase,2053013555631882655,electronics.smartphone,apple,1231.92,348a22e2-3e43-4396-9317-a2837b92247c,2019-11-01T03:25:40.000Z,view,2053013555631882655,electronics.smartphone,apple,1209.55,fcb1e15b-364b-472b-a64b-8fe19584ef5a
1005108,555394812,2019-10-31T00:33:37.000Z,purchase,2053013555631882655,electronics.smartphone,apple,1231.92,348a22e2-3e43-4396-9317-a2837b92247c,2019-11-01T03:25:56.000Z,purchase,2053013555631882655,electronics.smartphone,apple,1209.55,fcb1e15b-364b-472b-a64b-8fe19584ef5a
1005108,555394812,2019-10-31T00:33:37.000Z,purchase,2053013555631882655,electronics.smartphone,apple,1231.92,348a22e2-3e43-4396-9317-a2837b92247c,2019-11-01T03:26:24.000Z,view,2053013555631882655,electronics.smartphone,apple,1209.55,fcb1e15b-364b-472b-a64b-8fe19584ef5a
1005108,555394812,2019-10-31T00:33:37.000Z,purchase,2053013555631882655,electronics.smartphone,apple,1231.92,348a22e2-3e43-4396-9317-a2837b92247c,2019-11-02T02:57:17.000Z,view,2053013555631882655,electronics.smartphone,apple,1209.55,b675dce4-c802-4f7d-bc32-022e9b42b897
1005108,555394812,2019-10-31T00:33:37.000Z,purchase,2053013555631882655,electronics.smartphone,apple,1231.92,348a22e2-3e43-4396-9317-a2837b92247c,2019-11-03T02:07:16.000Z,view,2053013555631882655,electronics.smartphone,apple,1209.55,fe35a472-e76d-491d-a5f1-dfeb6b887d3e


In [0]:
from pyspark.sql.window import Window
import pyspark.sql.functions as F

w_session = Window.partitionBy("user_session").orderBy("event_time")

events_running = (df_oct
    .withColumn("running_spend", F.sum("price").over(w_session))
    .withColumn("running_event_count", F.count("*").over(w_session)))

display(events_running.limit(5))

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,running_spend,running_event_count
2019-10-14T19:03:55.000Z,view,1005003,2053013555631882655,electronics.smartphone,huawei,252.21,559841837,00001417-945d-4ab1-a69c-3e6a13b186f2,252.21,1
2019-10-26T10:51:18.000Z,view,1004026,2053013555631882655,electronics.smartphone,xiaomi,138.74,513409103,000089a1-ee47-49e1-9450-91f7c2e3155f,138.74,1
2019-10-17T07:37:27.000Z,view,7600336,2053013552821698803,null,tp-link,28.06,515841602,00009359-f7d4-49cc-b8eb-839702a78d3e,28.06,1
2019-10-17T07:38:44.000Z,view,7600336,2053013552821698803,null,tp-link,28.06,515841602,00009359-f7d4-49cc-b8eb-839702a78d3e,56.12,2
2019-10-17T07:39:53.000Z,view,7600336,2053013552821698803,null,tp-link,28.06,515841602,00009359-f7d4-49cc-b8eb-839702a78d3e,84.17999999999999,3


In [0]:
w_product_day = (Window.partitionBy("product_id", F.to_date("event_time")).orderBy("event_time"))

daily_running = df_nov.withColumn("daily_running_revenue",F.sum("price").over(w_product_day))

display(daily_running.limit(5))

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,daily_running_revenue
2019-11-20T00:25:48.000Z,view,1001618,2053013555631882655,electronics.smartphone,apple,501.66,553125129,810d2bbb-b74a-48fb-827b-a68dc4c0f838,501.66
2019-11-20T00:59:19.000Z,view,1001618,2053013555631882655,electronics.smartphone,apple,501.66,522545528,77ab2224-cde3-4696-8fc0-0cf96255cacd,1003.32
2019-11-20T02:07:48.000Z,view,1001618,2053013555631882655,electronics.smartphone,apple,501.66,557777417,c80c5c9f-57f6-4d46-b87d-3845844c6b14,1504.98
2019-11-20T02:32:52.000Z,view,1001618,2053013555631882655,electronics.smartphone,apple,501.66,512822949,711be8d5-58ad-4fb3-a497-5f465b494c75,2006.64
2019-11-20T02:54:00.000Z,view,1001618,2053013555631882655,electronics.smartphone,apple,501.66,558506254,152c8468-c2eb-4044-ac94-b0346a16af43,2508.3


In [0]:
events_features = (df_oct
    .withColumn("event_date", F.to_date("event_time"))
    .withColumn("hour", F.hour("event_time"))
    .withColumn("day_of_week", F.dayofweek("event_time"))
    .withColumn("is_weekend", F.dayofweek("event_time").isin([1,7])))
display(events_features.limit(5))

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,event_date,hour,day_of_week,is_weekend
2019-10-01T00:00:00.000Z,view,44600062,2103807459595387724,null,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c,2019-10-01,0,3,false
2019-10-01T00:00:00.000Z,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.2,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc,2019-10-01,0,3,false
2019-10-01T00:00:01.000Z,view,17200506,2053013559792632471,furniture.living_room.sofa,null,543.1,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8,2019-10-01,0,3,false
2019-10-01T00:00:01.000Z,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713,2019-10-01,0,3,false
2019-10-01T00:00:04.000Z,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d,2019-10-01,0,3,false


In [0]:
events_features = events_features.withColumn(
    "price_bucket",
    F.when(F.col("price") < 20, "low")
     .when((F.col("price") >= 20) & (F.col("price") < 100), "medium")
     .otherwise("premium")
)
display(events_features.limit(10))

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,event_date,hour,day_of_week,is_weekend,price_bucket
2019-10-01T00:00:00.000Z,view,44600062,2103807459595387724,null,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c,2019-10-01,0,3,false,medium
2019-10-01T00:00:00.000Z,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.2,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc,2019-10-01,0,3,false,medium
2019-10-01T00:00:01.000Z,view,17200506,2053013559792632471,furniture.living_room.sofa,null,543.1,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8,2019-10-01,0,3,false,premium
2019-10-01T00:00:01.000Z,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713,2019-10-01,0,3,false,premium
2019-10-01T00:00:04.000Z,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d,2019-10-01,0,3,false,premium
2019-10-01T00:00:05.000Z,view,1480613,2053013561092866779,computers.desktop,pulser,908.62,512742880,0d0d91c2-c9c2-4e81-90a5-86594dec0db9,2019-10-01,0,3,false,premium
2019-10-01T00:00:08.000Z,view,17300353,2053013553853497655,null,creed,380.96,555447699,4fe811e9-91de-46da-90c3-bbd87ed3a65d,2019-10-01,0,3,false,premium
2019-10-01T00:00:08.000Z,view,31500053,2053013558031024687,null,luminarc,41.16,550978835,6280d577-25c8-4147-99a7-abc6048498d6,2019-10-01,0,3,false,medium
2019-10-01T00:00:10.000Z,view,28719074,2053013565480109009,apparel.shoes.keds,baden,102.71,520571932,ac1cd4e5-a3ce-4224-a2d7-ff660a105880,2019-10-01,0,3,false,premium
2019-10-01T00:00:11.000Z,view,1004545,2053013555631882655,electronics.smartphone,huawei,566.01,537918940,406c46ed-90a4-4787-a43b-59a410c1a5fb,2019-10-01,0,3,false,premium


In [0]:
w_sess = Window.partitionBy("user_session")

events_features = events_features.withColumn(
    "session_start", F.min("event_time").over(w_sess)
).withColumn(
    "session_end", F.max("event_time").over(w_sess)
).withColumn(
    "session_duration_sec",
    F.col("session_end").cast("long") - F.col("session_start").cast("long")
)

display(events_features.limit(10))

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,event_date,hour,day_of_week,is_weekend,price_bucket,session_start,session_end,session_duration_sec
2019-10-31T06:23:12.000Z,view,1005115,2053013555631882655,electronics.smartphone,apple,955.84,513782162,00000056-a206-40dd-b174-a072550fa38c,2019-10-31,6,5,false,premium,2019-10-31T06:23:12.000Z,2019-10-31T06:31:25.000Z,493
2019-10-31T06:23:52.000Z,view,1005105,2053013555631882655,electronics.smartphone,apple,1349.46,513782162,00000056-a206-40dd-b174-a072550fa38c,2019-10-31,6,5,false,premium,2019-10-31T06:23:12.000Z,2019-10-31T06:31:25.000Z,493
2019-10-31T06:25:30.000Z,view,1005105,2053013555631882655,electronics.smartphone,apple,1349.46,513782162,00000056-a206-40dd-b174-a072550fa38c,2019-10-31,6,5,false,premium,2019-10-31T06:23:12.000Z,2019-10-31T06:31:25.000Z,493
2019-10-31T06:25:52.000Z,view,5100816,2053013553375346967,null,xiaomi,29.6,513782162,00000056-a206-40dd-b174-a072550fa38c,2019-10-31,6,5,false,medium,2019-10-31T06:23:12.000Z,2019-10-31T06:31:25.000Z,493
2019-10-31T06:26:58.000Z,view,1004858,2053013555631882655,electronics.smartphone,samsung,131.53,513782162,00000056-a206-40dd-b174-a072550fa38c,2019-10-31,6,5,false,premium,2019-10-31T06:23:12.000Z,2019-10-31T06:31:25.000Z,493
2019-10-31T06:28:21.000Z,view,1005104,2053013555631882655,electronics.smartphone,apple,993.27,513782162,00000056-a206-40dd-b174-a072550fa38c,2019-10-31,6,5,false,premium,2019-10-31T06:23:12.000Z,2019-10-31T06:31:25.000Z,493
2019-10-31T06:28:47.000Z,view,1003312,2053013555631882655,electronics.smartphone,apple,717.64,513782162,00000056-a206-40dd-b174-a072550fa38c,2019-10-31,6,5,false,premium,2019-10-31T06:23:12.000Z,2019-10-31T06:31:25.000Z,493
2019-10-31T06:31:25.000Z,view,1005212,2053013555631882655,electronics.smartphone,samsung,193.39,513782162,00000056-a206-40dd-b174-a072550fa38c,2019-10-31,6,5,false,premium,2019-10-31T06:23:12.000Z,2019-10-31T06:31:25.000Z,493
2019-10-10T04:34:05.000Z,view,1004249,2053013555631882655,electronics.smartphone,apple,769.62,557600066,00003ba6-7f01-4434-9e67-42568953f3a5,2019-10-10,4,5,false,premium,2019-10-10T04:34:05.000Z,2019-10-10T04:35:08.000Z,63
2019-10-10T04:34:57.000Z,view,1004243,2053013555631882655,electronics.smartphone,apple,733.25,557600066,00003ba6-7f01-4434-9e67-42568953f3a5,2019-10-10,4,5,false,premium,2019-10-10T04:34:05.000Z,2019-10-10T04:35:08.000Z,63


In [0]:
events_features = events_features.withColumn(
    "engagement_score",
    F.when(F.col("event_type") == "view", 1)
     .when(F.col("event_type") == "cart", 3)
     .when(F.col("event_type") == "purchase", 5)
     .otherwise(0)
)
display(events_features.limit(10))

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,event_date,hour,day_of_week,is_weekend,price_bucket,session_start,session_end,session_duration_sec,engagement_score
2019-10-31T06:23:12.000Z,view,1005115,2053013555631882655,electronics.smartphone,apple,955.84,513782162,00000056-a206-40dd-b174-a072550fa38c,2019-10-31,6,5,false,premium,2019-10-31T06:23:12.000Z,2019-10-31T06:31:25.000Z,493,1
2019-10-31T06:23:52.000Z,view,1005105,2053013555631882655,electronics.smartphone,apple,1349.46,513782162,00000056-a206-40dd-b174-a072550fa38c,2019-10-31,6,5,false,premium,2019-10-31T06:23:12.000Z,2019-10-31T06:31:25.000Z,493,1
2019-10-31T06:25:30.000Z,view,1005105,2053013555631882655,electronics.smartphone,apple,1349.46,513782162,00000056-a206-40dd-b174-a072550fa38c,2019-10-31,6,5,false,premium,2019-10-31T06:23:12.000Z,2019-10-31T06:31:25.000Z,493,1
2019-10-31T06:25:52.000Z,view,5100816,2053013553375346967,null,xiaomi,29.6,513782162,00000056-a206-40dd-b174-a072550fa38c,2019-10-31,6,5,false,medium,2019-10-31T06:23:12.000Z,2019-10-31T06:31:25.000Z,493,1
2019-10-31T06:26:58.000Z,view,1004858,2053013555631882655,electronics.smartphone,samsung,131.53,513782162,00000056-a206-40dd-b174-a072550fa38c,2019-10-31,6,5,false,premium,2019-10-31T06:23:12.000Z,2019-10-31T06:31:25.000Z,493,1
2019-10-31T06:28:21.000Z,view,1005104,2053013555631882655,electronics.smartphone,apple,993.27,513782162,00000056-a206-40dd-b174-a072550fa38c,2019-10-31,6,5,false,premium,2019-10-31T06:23:12.000Z,2019-10-31T06:31:25.000Z,493,1
2019-10-31T06:28:47.000Z,view,1003312,2053013555631882655,electronics.smartphone,apple,717.64,513782162,00000056-a206-40dd-b174-a072550fa38c,2019-10-31,6,5,false,premium,2019-10-31T06:23:12.000Z,2019-10-31T06:31:25.000Z,493,1
2019-10-31T06:31:25.000Z,view,1005212,2053013555631882655,electronics.smartphone,samsung,193.39,513782162,00000056-a206-40dd-b174-a072550fa38c,2019-10-31,6,5,false,premium,2019-10-31T06:23:12.000Z,2019-10-31T06:31:25.000Z,493,1
2019-10-10T04:34:05.000Z,view,1004249,2053013555631882655,electronics.smartphone,apple,769.62,557600066,00003ba6-7f01-4434-9e67-42568953f3a5,2019-10-10,4,5,false,premium,2019-10-10T04:34:05.000Z,2019-10-10T04:35:08.000Z,63,1
2019-10-10T04:34:57.000Z,view,1004243,2053013555631882655,electronics.smartphone,apple,733.25,557600066,00003ba6-7f01-4434-9e67-42568953f3a5,2019-10-10,4,5,false,premium,2019-10-10T04:34:05.000Z,2019-10-10T04:35:08.000Z,63,1


In [0]:
revenue = df_oct.filter(F.col("event_type") == "purchase") \
    .groupBy("product_id") \
    .agg(F.sum("price").alias("revenue")) \
    .orderBy(F.desc("revenue")).limit(5)
display(revenue)

product_id,revenue
1005115,2.0625574319999967E7
1005105,1.1445354689999992E7
1005135,7086522.1299999915
1004249,6815294.620000016
1002544,5603193.590000012


In [0]:
window = Window.partitionBy("user_id").orderBy("event_time")
running_per_user = df_oct.withColumn("cumulative_events", F.count("*").over(window))
display(running_per_user.limit(10))

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,cumulative_events
2019-10-09T10:30:19.000Z,view,17301541,2053013553853497655,null,null,162.17,205053188,e1eadbc6-aef5-4cff-bb1f-07d2b983a26e,1
2019-10-09T10:30:44.000Z,view,17301541,2053013553853497655,null,null,162.17,205053188,e1eadbc6-aef5-4cff-bb1f-07d2b983a26e,2
2019-10-07T06:23:01.000Z,view,16200119,2053013556344914381,kids.fmcg.diapers,moony,18.47,222907508,cb653adc-46a2-4d90-9e34-5bdfb2be30ce,1
2019-10-07T06:26:23.000Z,view,16200162,2053013556344914381,kids.fmcg.diapers,moony,18.47,222907508,cb653adc-46a2-4d90-9e34-5bdfb2be30ce,2
2019-10-08T14:29:09.000Z,view,6200883,2053013552293216471,appliances.environment.air_heater,elenberg,46.31,244673419,e2f0524c-bfc4-4c69-b93a-56f983027af3,1
2019-10-12T10:15:48.000Z,view,17300355,2053013553853497655,null,creed,240.16,257849716,71e76013-465a-4644-b82f-ab7fc64c9e95,1
2019-10-22T22:05:40.000Z,view,3900896,2053013552326770905,appliances.environment.water_heater,klima,77.2,266203246,c83d6f3d-2973-411f-8180-e476e65bc54c,1
2019-10-24T01:14:36.000Z,view,3900896,2053013552326770905,appliances.environment.water_heater,klima,77.2,266203246,56944410-059f-4f08-939b-867b3e060741,2
2019-10-06T11:29:22.000Z,view,22700574,2053013556168753601,null,null,88.81,278272605,e4cd7037-61d8-461c-ba6f-4314f0fb9a6f,1
2019-10-06T11:30:58.000Z,view,22700129,2053013556168753601,null,stels,66.93,278272605,0a20874c-c88c-4628-a6eb-886784b61d19,2


In [0]:
from pyspark.sql import functions as F

df_conversion_rate = (
    df_nov.groupBy("category_code")
    .agg(
        F.sum(F.when(F.col("event_type") == "purchase", 1).otherwise(0)).alias("purchase"),
        F.sum(F.when(F.col("event_type") == "view", 1).otherwise(0)).alias("view")
    )
    .withColumn(
        "conversion_rate",
        (F.col("purchase") / F.col("view")) * 100
    )
)
display(df_conversion_rate.limit(10))

category_code,purchase,view,conversion_rate
stationery.cartrige,191,11943,1.5992631667085322
electronics.video.tv,30274,2071305,1.461590639717473
accessories.wallet,366,68909,0.5311352653499543
appliances.kitchen.juicer,733,71894,1.0195565693938298
null,234218,20837460,1.1240237533749315
construction.tools.welding,1119,141960,0.7882502113271344
appliances.environment.air_heater,3583,269283,1.3305704407630634
country_yard.furniture.hammok,4,1567,0.2552648372686663
apparel.shoes,10140,1836676,0.5520843088274687
electronics.audio.microphone,489,44645,1.0953074252435884
